In [ ]:
using Plots
gr()

using TensorFlow

## Problem 2 from https://arxiv.org/pdf/physics/9705023.pdf


$$\dfrac{du}{dt}+\dfrac{1}{5}u=e^{-\frac{t}{5}}cos(t)$$
 

With $u(0)=0$
  and $t\in[0,2]$
 

Analytic solution is $$u_{ana}(t)=e^{-\frac{t}{5}}sin(t)$$
 

Trail solution is $$u_{trial}(t)=tN(t,\tilde{p})$$
 

In [ ]:

sess=Session(Graph())
hl_width=64
a=10
@tf begin #Automatically name nodes based on RHS
    t = placeholder(Float32; shape=[-1])
    tt = expand_dims(t, 2) #make it a matrix
    
    w1 = get_variable([1,hl_width], Float32)
    b1 = get_variable([hl_width], Float32)
    w2 = get_variable([hl_width,1], Float32)
    
    #$u_{trial} = t*N(t, p̃). p̃={w1, b1, w2}
    net1 = nn.sigmoid(tt*w1 + b1)*w2
    u = tt.*net1
    
    
    du_dt = gradients(u, tt)
    deq_lhs = du_dt + u/5
    deq_rhs = exp(-tt/5).*cos(tt)
        
    
    loss = reduce_sum((deq_lhs - deq_rhs).^2)
    opt = train.minimize(train.AdamOptimizer(), loss)
end


In [ ]:
t_obs = collect(0.0:0.02:2.0)

run(sess, global_variables_initializer())
for ii in 1:1000
    _, loss_o = run(sess, [opt, loss], Dict(t=>t_obs))
    if ii%50 == 1
        println(loss_o)
    end
end

In [ ]:
u_net = run(sess, u, Dict(t=>t_obs))
u_ana = exp(-t_obs/5).*sin(t_obs)


plot(t_obs, [u_net u_ana]; label=["u_net", "u_ana"])
